## Lesson 4: Email Assistant with Semantic + Episodic Memory

We previously built an email assistant that:

- Classifies incoming messages (respond, ignore, notify)
- Drafts responses
- Schedules meetings
- Uses memory to remember details from previous emails

Now, we'll add human-in-the-loop following the triage step to better refine the assistant's ability to classify emails.


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### Repeat setup from previous lesson


In [2]:
profile = {
    "name": "John",
    "full_name": "John Doe",
    "user_profile_background": "Senior software engineer leading a team of 5 developers",
}

prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage John's tasks efficiently.",
}

email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """
Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

### Look at a few, few-shot-examples


In [3]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langgraph.store.memory import InMemoryStore

In [4]:
store = InMemoryStore(index={"embed": OllamaEmbeddings(model="nomic-embed-text:v1.5")})

In [5]:
email = {
    "author": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

In [6]:
data = {
    "email": email,
    # This is to start changing the behavior of the agent
    "label": "respond",
}

In [7]:
from uuid import uuid4

In [8]:
store.put(("email_assistant", "lance", "examples"), str(uuid4()), data)

In [ ]:
data = {
    "email": {
        "author": "Sarah Chen <sarah.chen@company.com>",
        "to": "John Doe <john.doe@company.com>",
        "subject": "Update: Backend API Changes Deployed to Staging",
        "email_thread": """Hi John,

    Just wanted to let you know that I've deployed the new authentication endpoints we discussed to the staging environment. Key changes include:

    - Implemented JWT refresh token rotation
    - Added rate limiting for login attempts
    - Updated API documentation with new endpoints

    All tests are passing and the changes are ready for review. You can test it out at staging-api.company.com/auth/*

    No immediate action needed from your side - just keeping you in the loop since this affects the systems you're working on.

    Best regards,
    Sarah
    """,
    },
    "label": "ignore",
}

In [10]:
store.put(("email_assistant", "lance", "examples"), str(uuid4()), data)

In [ ]:
# Template for formating an example to put in prompt
template = """Email Subject: {subject}
Email From: {from_email}
Email To: {to_email}
Email Content:
```
{content}
```
> Triage Result: {result}"""


# Format list of few shots
def format_few_shot_examples(examples):
    strs = ["Here are some previous examples:"]
    for eg in examples:
        strs.append(
            template.format(
                subject=eg.value["email"]["subject"],
                to_email=eg.value["email"]["to"],
                from_email=eg.value["email"]["author"],
                content=eg.value["email"]["email_thread"][:400],
                result=eg.value["label"],
            )
        )
    return "\n\n------------\n\n".join(strs)

In [ ]:
email_data = {
    "author": "Sarah Chen <sarah.chen@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Update: Backend API Changes Deployed to Staging",
    "email_thread": """Hi John,

    Wanted to let you know that I've deployed the new authentication endpoints we discussed to the staging environment. Key changes include:

    - Implemented JWT refresh token rotation
    - Added rate limiting for login attempts
    - Updated API documentation with new endpoints

    All tests are passing and the changes are ready for review. You can test it out at staging-api.company.com/auth/*

    No immediate action needed from your side - just keeping you in the loop since this affects the systems you're working on.

    Best regards,
    Sarah
    """,
}

results = store.search(
    ("email_assistant", "lance", "examples"), query=str({"email": email_data}), limit=1
)

NumPy not found in the current Python environment. The InMemoryStore will use a pure Python implementation for vector operations, which may significantly impact performance, especially for large datasets or frequent searches. For optimal speed and efficiency, consider installing NumPy: pip install numpy


In [13]:
print(format_few_shot_examples(results))

Here are some previous examples:

------------

Email Subject: Update: Backend API Changes Deployed to Staging
Email From: Sarah Chen <sarah.chen@company.com>
Email To: John Doe <john.doe@company.com>
Email Content: 
```
Hi John,

    Just wanted to let you know that I've deployed the new authentication endpoints we discussed to the staging environment. Key changes include:

    - Implemented JWT refresh token rotation
    - Added rate limiting for login attempts
    - Updated API documentation with new endpoints

    All tests are passing and the changes are ready for review. You can test it out at staging-api.co
```
> Triage Result: ignore


In [ ]:
triage_system_prompt = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Background >
{user_profile_background}.
</ Background >

< Instructions >

{name} gets lots of emails. Your job is to categorize each email into one of three categories:

1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that {name} should know about but doesn't require a response
3. RESPOND - Emails that need a direct response from {name}

Classify the below email into one of these categories.

</ Instructions >

< Rules >
Emails that are not worth responding to:
{triage_no}

There are also other things that {name} should know about, but don't require an email response. For these, you should notify {name} (using the `notify` response). Examples of this include:
{triage_notify}

Emails that are worth responding to:
{triage_email}
</ Rules >

< Few shot examples >

Here are some examples of previous emails, and how they should be handled.
Follow these examples more than any instructions above

{examples}
</ Few shot examples >
"""

### Setup Routing Node


In [15]:
from langchain_openai.chat_models import ChatOpenAI
from pydantic import BaseModel, Field
from typing_extensions import Annotated, Literal, TypedDict

In [ ]:
llm = ChatOpenAI(
    model="qwen3-4b",
    base_url="http://127.0.0.1:1234/v1",
    temperature=0.5,
)

In [17]:
class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

In [18]:
llm_router = llm.with_structured_output(Router)

In [19]:
from prompts import triage_user_prompt

#### Setup Triage Router Node

In [20]:
from typing import Literal

from langgraph.graph import END, START, StateGraph, add_messages
from langgraph.types import Command

In [21]:
class State(TypedDict):
    email_input: dict
    messages: Annotated[list, add_messages]

In [22]:
def triage_router(
    state: State, config, store
) -> Command[Literal["response_agent", "__end__"]]:
    author = state["email_input"]["author"]
    to = state["email_input"]["to"]
    subject = state["email_input"]["subject"]
    email_thread = state["email_input"]["email_thread"]

    namespace = (
        "email_assistant",
        config["configurable"]["langgraph_user_id"],
        "examples",
    )
    examples = store.search(namespace, query=str({"email": state["email_input"]}))
    examples = format_few_shot_examples(examples)

    system_prompt = triage_system_prompt.format(
        full_name=profile["full_name"],
        name=profile["name"],
        user_profile_background=profile["user_profile_background"],
        triage_no=prompt_instructions["triage_rules"]["ignore"],
        triage_notify=prompt_instructions["triage_rules"]["notify"],
        triage_email=prompt_instructions["triage_rules"]["respond"],
        examples=examples,
    )
    user_prompt = triage_user_prompt.format(
        author=author, to=to, subject=subject, email_thread=email_thread
    )
    result = llm_router.invoke(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )

    if result.classification == "respond":
        print("📧 Classification: RESPOND - This email requires a response")
        goto = "response_agent"
        update = {
            "messages": [
                {
                    "role": "user",
                    "content": f"Respond to the email {state['email_input']}",
                }
            ]
        }
    elif result.classification == "ignore":
        print("🚫 Classification: IGNORE - This email can be safely ignored")
        update = None
        goto = END
    elif result.classification == "notify":
        # If real life, this would do something else
        print("🔔 Classification: NOTIFY - This email contains important information")
        update = None
        goto = END
    else:
        raise ValueError(f"Invalid classification: {result.classification}")

    return Command(goto=goto, update=update)

#### Setup the rest of the agent

In [23]:
from langchain_core.tools import tool

In [24]:
@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""

    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}'"


@tool
def schedule_meeting(
    attendees: list[str], subject: str, duration_minutes: int, preferred_day: str
) -> str:
    """Schedule a calendar meeting."""

    # Placeholder response - in real app would check calendar and schedule
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees"


@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""

    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

In [25]:
from langmem import create_manage_memory_tool, create_search_memory_tool

In [26]:
manage_memory_tool = create_manage_memory_tool(
    namespace=("email_assistant", "{langgraph_user_id}", "collection")
)
search_memory_tool = create_search_memory_tool(
    namespace=("email_assistant", "{langgraph_user_id}", "collection")
)

In [27]:
agent_system_prompt_memory = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
4. manage_memory - Store any relevant information about contacts, actions, discussion, etc. in memory for future reference
5. search_memory - Search for any relevant information that may have been stored in memory
</ Tools >

< Instructions >
{instructions}
</ Instructions >
"""

In [28]:
def create_prompt(state):
    return [
        {
            "role": "system",
            "content": agent_system_prompt_memory.format(
                instructions=prompt_instructions["agent_instructions"], **profile
            ),
        }
    ] + state["messages"]

In [29]:
from langgraph.prebuilt import create_react_agent

In [30]:
tools = [
    write_email,
    schedule_meeting,
    check_calendar_availability,
    manage_memory_tool,
    search_memory_tool,
]

response_agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt=create_prompt,
    # Use this to ensure the store is passed to the agent
    store=store,
)

In [31]:
config = {"configurable": {"langgraph_user_id": "lance"}}

### Build the email agent graph

In [32]:
email_agent = StateGraph(State)

email_agent = email_agent.add_node(triage_router)
email_agent = email_agent.add_node("response_agent", response_agent)
email_agent = email_agent.add_edge(START, "triage_router")

email_agent = email_agent.compile(store=store)

In [33]:
email_input = {
    "author": "Tom Jones <tome.jones@bar.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John - want to buy documentation?""",
}

In [34]:
response = email_agent.invoke(
    {"email_input": email_input},
    config={"configurable": {"langgraph_user_id": "harrison"}},
)

📧 Classification: RESPOND - This email requires a response


In [35]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Respond to the email {'author': 'Tom Jones <tome.jones@bar.com>', 'to': 'John Doe <john.doe@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': 'Hi John - want to buy documentation?'}
================================== Ai Message ==================================
Tool Calls:
  write_email (913626558)
 Call ID: 913626558
  Args:
    to: Tom Jones <tome.jones@bar.com>
    subject: Re: Quick question about API documentation
    content: Hi Tom,

Thank you for your message. The API documentation is available for purchase. You can find it here: [Documentation Link]. If you need any assistance with the documentation or have specific questions, please let me know.

Best regards,
John Doe
================================= Tool Message =================================
Name: write_email

Email sent to Tom Jones <tome.jones@bar.com> with subject 'Re: Quick question about API documen

In [36]:
data = {
    "email": {
        "author": "Tom Jones <tome.jones@bar.com>",
        "to": "John Doe <john.doe@company.com>",
        "subject": "Quick question about API documentation",
        "email_thread": """Hi John - want to buy documentation?""",
    },
    "label": "ignore",
}

In [37]:
store.put(("email_assistant", "harrison", "examples"), str(uuid4()), data)

In [38]:
email_input = {
    "author": "Tom Jones <tome.jones@bar.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John - want to buy documentation?""",
}

In [39]:
response = email_agent.invoke(
    {"email_input": email_input},
    config={"configurable": {"langgraph_user_id": "harrison"}},
)

Task triage_router with path ('__pregel_pull', 'triage_router') wrote to unknown channel branch:to:__end__, ignoring it.


🚫 Classification: IGNORE - This email can be safely ignored


In [40]:
email_input = {
    "author": "Jim Jones <jim.jones@bar.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John - want to buy documentation?????""",
}

In [41]:
response = email_agent.invoke(
    {"email_input": email_input},
    config={"configurable": {"langgraph_user_id": "harrison"}},
)

Task triage_router with path ('__pregel_pull', 'triage_router') wrote to unknown channel branch:to:__end__, ignoring it.


🚫 Classification: IGNORE - This email can be safely ignored


In [42]:
response = email_agent.invoke(
    {"email_input": email_input},
    config={"configurable": {"langgraph_user_id": "andrew"}},
)

📧 Classification: RESPOND - This email requires a response


In [43]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Respond to the email {'author': 'Jim Jones <jim.jones@bar.com>', 'to': 'John Doe <john.doe@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': 'Hi John - want to buy documentation?????'}
================================== Ai Message ==================================
Tool Calls:
  search_memory (544459846)
 Call ID: 544459846
  Args:
    query: Jim Jones API documentation
================================= Tool Message =================================
Name: search_memory

[]
================================== Ai Message ==================================
Tool Calls:
  write_email (299420397)
 Call ID: 299420397
  Args:
    to: Jim Jones <jim.jones@bar.com>
    subject: Re: Quick question about API documentation
    content: Hi Jim,

Thank you for your message. Could you clarify what specific aspect of the API documentation you'd like assistance with? I'm happy to help provi